In [139]:
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
from src.vastdb import VastDB
from src.manager import DbManager
from src.fetch import fetch
from src.preprocess import preprocess, split_raw
from src import const
from src.tables import get_machines

import logging


logging.basicConfig(format=const.LOG_FORMAT,
                    level=logging.DEBUG,
                    datefmt='%d-%m-%Y %I:%M:%S')

# logger = 
# logger.setLevel(logging.DEBUG)

In [141]:
vast = VastDB('../data/test.db')
vast.connect()

[10-10-2023 01:57:32] [INFO] Connected to the database '../data/test.db'


In [142]:
vast.dbm.get_tables()

['timestamp_idx',
 'machine_host_map',
 'online_ts',
 'online_snp',
 'hardware_ts',
 'hardware_snp',
 'cpu_ram_ts',
 'cpu_ram_snp',
 'disk_ts',
 'disk_snp',
 'eod_ts',
 'eod_snp',
 'reliability_ts',
 'reliability_snp',
 'cost_ts',
 'cost_snp',
 'rent_ts',
 'rent_snp',
 'avg_ts',
 'avg_snp']

In [143]:
info = vast.dbm.get_tbl_info('tmp_machines')
info
# 'host_id' in info['name']

,name,type,notnull,dflt_value,pk
cid,,,,,


In [144]:
vast.init_tables()

In [145]:
vast.create_tables()

In [146]:
vast.dbm.get_tables()

['timestamp_idx',
 'machine_host_map',
 'online_ts',
 'online_snp',
 'hardware_ts',
 'hardware_snp',
 'cpu_ram_ts',
 'cpu_ram_snp',
 'disk_ts',
 'disk_snp',
 'eod_ts',
 'eod_snp',
 'reliability_ts',
 'reliability_snp',
 'cost_ts',
 'cost_snp',
 'rent_ts',
 'rent_snp',
 'avg_ts',
 'avg_snp']

In [147]:
vast.dbm.get_db_stats()

,name,pages,size,percentage
0,total,399,1.56,100.00
1,avg_snp,84,0.33,21.03
2,hardware_snp,68,0.27,17.03
3,eod_snp,53,0.21,13.27
4,hardware_ts,39,0.15,9.77
5,eod_ts,32,0.13,8.01
6,cost_snp,17,0.07,4.26
7,cost_ts,14,0.05,3.51
8,online_snp,10,0.04,2.50
9,reliability_ts,10,0.04,2.50


In [148]:
# for tbl in vast.dbm.get_tables():
#     vast.dbm.delete_table(tbl)

In [149]:
machines = get_machines()

[10-10-2023 01:57:33] [DEBUG] Starting new HTTPS connection (1): 500.farm:443
[10-10-2023 01:57:34] [DEBUG] https://500.farm:443 "GET /vastai-exporter/machines HTTP/1.1" 200 None


In [150]:
vast.create_tmp_tables(machines)

[10-10-2023 01:57:35] [DEBUG] Temporary tables created: 'tmp_machines'


In [151]:
# vast.dbm.table_to_df('tmp_machines')

,bw_nvlink,client_run_time,compute_cap,cpu_cores,cpu_name,cpu_ram,credit_discount_max,cuda_max_good,direct_port_count,disk_bw,...,reliability,score,static_ip,storage_cost,total_flops,verification,vram_costperhour,timestamp,isp,country
0,0,1.1,860,20,Xeon® E5-2630 v4,252,400,12.0,1020,514,...,5727,9,1,150,35,0,8.951823e-06,1696939021,Artnet Sp. z o.o.,PL
1,0,1.1,860,24,Xeon® E5-2680 v3,63,400,12.2,101,2421,...,6397,16,1,50,12,0,3.255208e-06,1696939021,CenturyLink,US
2,0,1.1,860,24,Xeon® E5-2680 v3,63,400,12.2,102,2270,...,6769,50,1,50,13,0,3.255208e-06,1696939021,CenturyLink,US
3,0,1.1,860,24,Xeon® E5-2680 v3,63,400,12.0,101,179,...,7141,58,1,50,13,0,3.255208e-06,1696939021,CenturyLink,US
4,0,1.1,860,12,AMD Ryzen 5 3600 6-Core Processor,16,400,12.0,100,1689,...,7220,188,1,150,59,0,9.765625e-07,1696939021,iFiber Communications Corp.,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,0,1.1,610,40,Xeon® E5-2658 v2,63,100,12.0,1000,125,...,9894,50,1,150,37,1,6.713867e-06,1696939021,State Enterprise Scientific and Telecommunication,UA
1728,0,1.1,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1524,...,9963,49,1,150,49,2,6.713867e-06,1696939021,State Enterprise Scientific and Telecommunication,UA
1729,0,1.1,610,32,Xeon® E5-2650 v2,63,100,12.0,1000,1634,...,9960,51,1,150,49,1,6.713867e-06,1696939021,State Enterprise Scientific and Telecommunication,UA
1730,0,1.1,750,32,AMD Ryzen Threadripper 1950X 16-Core Processor,63,400,11.7,1000,1804,...,9979,48,1,150,61,1,1.331676e-05,1696939021,Atlantic Broadband,US


In [152]:
vast.update_tables()

1059

In [153]:
vast.dbm.commit()

In [155]:
vast.close()

[10-10-2023 02:18:03] [DEBUG] Closed connection to the database '../data/test.db'.


In [138]:
db.dbm.create_table('tmp_machines', ['host_id', 'machine_id', 'timestamp'])
db.dbm.insert('tmp_machines',
              [[1, 2, 16],
               [1, 3, 16],
               [2, 4, 16]])

NameError: name 'db' is not defined

In [ ]:
db.dbm.create_table('tmp_offers', ['machine_id', 'id', 'num_gpus', 'timestamp'])
db.dbm.insert('tmp_offers', 
                  [[1, 1, 1, 16], 
                   [1, 2, 1, 16], 
                   [2, 3, 2, 16]])

In [ ]:
db.create_map_tables()
db.update_tables()
# db.dbm.commit()
db.dbm.get_db_stats()

In [ ]:
# db.dbm.table_to_df('host_mach_map')
# db.dbm.table_to_df('mach_offer_map')
db.dbm.table_to_df('chunks_map')

In [ ]:
db.update_tables()

In [ ]:
# db.connect()

for tbl in db.dbm.get_tables():
    db.dbm.delete_table(tbl)

db.close()

In [ ]:
raw = fetch()
preprocess(raw)
machines, offers = split_raw(raw)

In [ ]:
db.connect()
db.create_tmp_tables(machines, offers)
db.create_map_tables()

In [ ]:
db.dbm.get_db_stats()
# db.dbm.delete_table('int32')

In [ ]:
db.create_ts_index()

In [ ]:
db.update_tables()

In [ ]:
db.dbm.commit()

In [ ]:
db.dbm.get_db_stats()

In [ ]:
# db.dbm.table_to_df('mach_offer_map')
db.dbm.table_to_df('mach_offer_map').sort_values(by='machine_id')

In [ ]:
db.dbm.table_to_df('chunks_map').sort_values(by='id')

In [ ]:
db.dbm.table_to_df('tmp_offers').sort_values(by='id')

In [ ]:
db.close()